<a href="https://colab.research.google.com/github/RaianaSeixas/Ploblems_Optimization/blob/master/Problem_Gear_train_Ger_MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
############################### Importar códigos

!git clone https://github.com/RaianaSeixas/GERENCIADOR.git #clonar do Github

Cloning into 'GERENCIADOR'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 101 (delta 5), reused 0 (delta 0), pack-reused 85
Receiving objects: 100% (101/101), 44.09 KiB | 3.39 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [10]:
%cd
%cd /content/GERENCIADOR
!ls

/root
/content/GERENCIADOR
1_WOA_Main_R.ipynb    4_GWO_Main_R.ipynb    Enxame.py	 MUT.py
2_ABC_Main_R.ipynb    ABCMC.py		    Function.py  PSO.py
2_ABCMC_Main_R.ipynb  ABC.py		    GERENCIADOR  __pycache__
3_PSO_Main_R.ipynb    DE_jun_16_2019.ipynb  GWO.py	 WOA.py


In [0]:
'''3 bar Truss Function'''
def Fun(x):
  fx=(1/6.931-x[2]*x[1]/(x[0]*x[3]))**2
  return fx


In [0]:
import numpy as np
MIN=np.repeat(12,4)
MAX=np.repeat(60,4)

In [13]:
############################### Importar bibliotecas e iniciar a contagem do tempo
import time
#import matplotlib.pyplot as plt
from random import randint

############################### Importar códigos
'''
!git clone https://github.com/RaianaSeixas/GERENCIADOR.git #clonar do Github

# %cd
# %cd /content/GERENCIADOR
!ls
'''

############################### DADOS DE ENTRADA 
RUNS=10 #NÚMERO DE RODADAS
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

################################ FASE EXPLORAÇÃO 
ITE=300 #ITERACOES
NPAR=500 #AGENTES POR MÉTODO
MET=2 #NUM DE MÉTODOS FASE EXPLORAÇÃO 

################################## FASE MUTAÇÃO
itermut=300 #Iterações de mutação 
nw=200 #Agentes totais na fase de mutação
n=10 #qde de melhores agentes de tds os métodos coletados
nt=3 #qde de melhores agentes de cada método coletados (primeira iter)

################################### FASE CONVERGÊNCIA
ITE_G=200 #ITERAÇÕES
PTRADE=200 #NUM DE PARTÍCULAS A SERERM TROCADAS ENTRE OS MÉTODOS

W=0.5
C1=1
C2=1

##################################### IMPORTAR MÓDULOS
import Enxame
import MUT
#import GWO
import WOA
import ABC
import ABCMC
import PSO

start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

'''
def count(fn):
        def wrapper(*args, **kwargs):
            wrapper.called+= 1
            return fn(*args, **kwargs)
        wrapper.called= 0
        wrapper.__name__= fn.__name__
        return wrapper 

#FOBJ gera vetorialmente os valores com a funcao objetivo escalar
#em outras palavras gera para populacao o valor da funcao a ser otimizada

@count
def FOBJ(X,Fun):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj
'''
BEST_AGENTS=[]
for i in range(RUNS):
    
    ############################## BALEIA - WOA
    
    
    X=Enxame.Enxame(PAR,NPAR,MAX,MIN) # inicializa baleias
    
    BEST_WOA,FOBEST_WOA,XY_WOA,BEST_XY_WOA=WOA.WOA(ITE,PAR,NPAR,MAX,MIN,Fun,X) #Roda Algoritmo
    
    #print("\n","WOA _",NPAR,"Agentes _",ITE,"Iterações _","Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__,"\n")
    #print("BEST=",BEST_WOA,"  FOBEST=",FOBEST_WOA,"\n")
    #print("FOBEST=",FOBEST_WOA,"\n")
    
    XY_WOA_T=XY_WOA[0:PTRADE,]
    #print(XY_WOA_T)
    
    ############################## ABELHAS - ABC
    
    ntrail=10 #numero de buscas ate abandonar uma fonte de alimento
    
    X=Enxame.Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
    
    BEST_ABC,FOBEST_ABC,XY_ABC,BEST_XY_ABC=ABCMC.ABCOPtim(ITE,PAR,NPAR,MAX,MIN,Fun,X,ntrail)
    
    #print("\n","ABC _",NPAR,"Agentes _",ITE,"Iterações _","Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__,"\n")
    #print("BEST=",BEST_ABC,"  FOBEST=",FOBEST_ABC, "\n")
    
    BEST_XY_ABC=np.append(BEST_ABC,FOBEST_ABC)
   
    ############################ SOS Mutações
    #MBEST=np.vstack([BEST_XY_ABC,BEST_XY_GWO,BEST_XY_WOA,BEST_XY_PSO]) # reúne o melhor agente de cada método
    MBEST=np.vstack([BEST_XY_ABC,BEST_XY_WOA]) # reúne o melhor agente de cada método
    #print(MBEST)
    
    # GWO
    #X_GWO=MUT.GETX(XY_GWO,PAR)
    #Y_GWO=MUT.GETY(XY_GWO)
    
    # WOA
    X_WOA=MUT.GETX(XY_WOA,PAR)
    Y_WOA=MUT.GETY(XY_WOA)
    
    # PSO
    #X_PSO=MUT.GETX(XY_PSO,PAR)
    #Y_PSO=MUT.GETY(XY_PSO)
    
    # MUT3New GWO WOA PSO
    '''
    Xnew3=MUT.MUT3(X_GWO,MBEST,NPAR,PAR,MAX,MIN) # Aplica Mutação
    Ynew3=MUT.FOBJ(Xnew3,Fun) # Obter FO mutante
    XYm3_GWO=MUT.XYsort(Xnew3,Ynew3) #Junta e ordena tds os agentes
    #print('XYm_GWO3: ',XYm3_GWO)
    '''
    Xnew3=MUT.MUT3(X_WOA,MBEST,NPAR,PAR,MAX,MIN) # Aplica Mutação
    Ynew3=MUT.FOBJ(Xnew3,Fun) # Obter FO mutante
    XYm3_WOA=MUT.XYsort(Xnew3,Ynew3) #Junta e ordena tds os agentes
    #print('XYm3_WOA: ',XYm3_WOA)
    '''
    Xnew3=MUT.MUT3(X_PSO,MBEST,NPAR,PAR,MAX,MIN) # Aplica Mutação
    Ynew3=MUT.FOBJ(Xnew3,Fun) # Obter FO mutante
    XYm3_PSO=MUT.XYsort(Xnew3,Ynew3) #Junta e ordena tds os agentes
    #print('XYm3_PSO: ',XYm3_PSO)
    '''
    XYm3=np.vstack((BEST_XY_ABC,XYm3_WOA))
    XYm3=XYm3[0:nw]
    
    # MUT3new MAIN

    #######################################################################    
    XY=np.vstack((BEST_XY_ABC,XYm3_WOA[nt]))
    XY3BEST = XY[XY[:,-1].argsort()] #Ordena a partir de Y

    XYCOLECT=XY3BEST
    
    #Aplica intermut iterações e coleta n melhores agentes das iterações e fazer um sort no final 
    for i in range(itermut):      
        Xnew3=MUT.MUT3(XYm3,XY3BEST,nw,PAR,MAX,MIN) # Aplica Mutação
        Ynew3=MUT.FOBJ(Xnew3,Fun) # Obter FO mutante
        XYm3=MUT.XYsort(Xnew3,Ynew3) #Junta e ordena tds os agentes
        #print('XYm3: ',XYm3[0:nt])
        XY3BEST=XYm3[0:n]    
        XYCOLECT=np.vstack((XYCOLECT,XY3BEST))
    XYCOLECTst = XYCOLECT[XYCOLECT[:,-1].argsort()]
    
    #XY GERENCIADORES
    
    XY_G3new=np.vstack((MBEST,XYCOLECTst[0:(PTRADE*(MET-1))]))
    #print(XY_G3new)
    X_G3new=MUT.GETX(XY_G3new,PAR) #Extrai as primeiras colunas (valores de X)
       
    BEST_G,FOBEST_G,XY_G,BEST_XY_G=PSO.PSO(W,C1,C2,len(X_G3new),(ITE_G),PAR,MAX,MIN,Fun,X_G3new)
    
    BEST_AGENTS.append(BEST_XY_G)
    
    #print("\n","Gerenciador 3_NEW",len(X_G3new),"Agentes _",ITE_G,"Iterações _","Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__,"\n")
    #print("BEST=",BEST_G,"  FOBEST=",FOBEST_G,"\n")
    #print("FOBEST=",FOBEST_G,"\n")
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

print("\n","Gerenciador 3_NEW",len(X_G3new),"Agentes _",ITE_G,"Iterações _","Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print('Métodos simples:',NPAR,'Agentes_',ITE,'Iterações_','Dim=', PAR)
print(NPAR*(MET-1),'Agentes',itermut,'Iterações de mutação', '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1])
print('MIN__: ',MINFO)
print('MAX__: ',MAXFO)
print('STD__: ',STD)
print('BEST RESULT: ',BEST_AGENTS_ST[0])
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas')

#otimo mfo= 263.89597968
#BEST RESULT__: [  0.78864361   0.40833746 263.89584411]


 Gerenciador 3_NEW 202 Agentes _ 200 Iterações _ Min/Max= 12 / 60 _ Fun
Métodos simples: 500 Agentes_ 300 Iterações_ Dim= 4
500 Agentes 300 Iterações de mutação 

MÉDIA:  0.0
MIN__:  0.0
MAX__:  0.0
STD__:  0.0
BEST RESULT:  [37.65049205 20.15764598 13.76392898 51.07489463  0.        ]
Tempo Médio= 24.636 segundos ---
10 Rodadas


In [14]:
best=BEST_AGENTS_ST[0]
x1=round(best[0])
x2=round(best[1])
x3=round(best[2])
x4=round(best[3])
BEST=[x1,x2,x3,x4]
Fun(BEST)

3.980798957436708e-08

In [15]:
BEST_AGENTS_ST

array([[37.65049205, 20.15764598, 13.76392898, 51.07489463,  0.        ],
       [21.97236907, 12.99945659, 13.53987121, 55.52118737,  0.        ],
       [46.13020436, 16.724271  , 23.76167329, 59.70830421,  0.        ],
       [55.50975712, 14.72141185, 26.84789339, 49.34989685,  0.        ],
       [58.73993862, 16.68869965, 24.20801472, 47.66988277,  0.        ],
       [39.58193059, 16.10284436, 20.70700762, 58.38736343,  0.        ],
       [49.08084305, 13.93031605, 30.43326893, 59.86782196,  0.        ],
       [55.35553081, 12.5418678 , 37.55828219, 58.97973524,  0.        ],
       [33.81651897, 16.65320049, 17.03694658, 58.15090408,  0.        ],
       [26.34768668, 12.0174924 , 18.87634865, 59.67401441,  0.        ]])